In [1]:
import pandas as pd
import re
import jieba
import datetime

In [2]:
# 导入数据
df = pd.read_csv("data.csv",encoding="utf-8")
total_words = df['评论'].str.len().sum()
print(total_words)

4396378


In [3]:
# 清除非中文文本
df['评论'] = df['评论'].apply(lambda x: re.sub('[^\u4e00-\u9fff]', '', x))

In [4]:
# 读入停用词档，并去除停用词前后空白、换行符
stopwords=open('stopwords.txt',"r",encoding='UTF-8').readlines()
stopwords = [s.strip() for s in stopwords]
print('停用词数量：' + str(len(stopwords)))
# 分词
for i in range(len(df)):
    #1.从 DataFrame 逐条读入 清理后的新闻的内容
    content = df.评论[i]
    #2.对新闻的内容进行分词
    words = jieba.lcut(content)    
    #3.去除 停用词 及 单词
    word_cut_stopwords = words
    for w in stopwords:
        for j in range(len(word_cut_stopwords)-1, -1, -1):
            if ((len(str(word_cut_stopwords[j])) == 1) or \
                (word_cut_stopwords[j] == w)):
                word_cut_stopwords.pop(j) 

    df.loc[i, '分词后内容'] = ' '.join(word_cut_stopwords)

new_total_words = df['分词后内容'].str.len().sum()
print(new_total_words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\余明鑫\AppData\Local\Temp\jieba.cache


停用词数量：1621


Loading model cost 0.797 seconds.
Prefix dict has been built successfully.


2393101


In [5]:
# 删除空值的行
df['分词后内容'] = df['分词后内容'].replace('', None)
df = df.dropna(subset=['分词后内容'])
df = df.reset_index(drop=True)
df.describe()

,歌名,评论,分词后内容
count,165181,165181,165181
unique,3,121803,97646
top,孤独患者,加油,加油
freq,56084,4350,5183


In [ ]:
# 将全部分词后的内容存入 Excel 档
df.to_excel('data分词后.xlsx', header=True)